In [2]:
import pandas as pd
import numpy as np

This script shows the basic stats for the annotation
1. We count the percentage of positive/all posts (neg_per), negative/all posts (pos_per), mixed/all posts  (mix_per), neutral/ /all posts (neg_per), negative/positive (pos_neg) 
2. Recoded CESD data
3. Describe the data set
4. Correlation of all variables
Here we see that the percentage of mixed post has a high correlation with cesd score, but it might be biased because the number of mixed post is only 88 Belows shows the stats of people have mixed posts only 44 people have mixed posts, among 88 posts, 1/8 of the posts belong to the same person


In [9]:
time = pd.read_csv('../important_data/twoM_newLabels.csv')
time  = time[['text','id','negative_ny','userid']]
time.shape

(4099, 4)

In [10]:
print(time.shape)
time.to_csv('../data/twoM_newLabels.csv')
time[1:5]

(4099, 4)


,text,id,negative_ny,userid
1,I don't get many things right the first time. ...,277,3,e9970fe8848e9505b52d414d81611de8
2,Three days off! I'm so excited I woke up at 6a...,4120,3,0e3fdb89c7706b13891170ab2a9017c7
3,"/Is this a happy story?/"" ....Partly.""",4121,3,1789589a6d2bba0f09d768f8f3445be5
4,omgeez......sooo tired....it was like everyone...,4122,3,23cbbe03a89344129e1a6d8419a66d23


In [11]:
#merge with time variable 
time['negative_ny'].value_counts()

2    2002
1    1209
4     768
3      88
5      32
Name: negative_ny, dtype: int64

1 negative
2 positive
3 mixed
4 neutral
5 foreign language


percentage of negative post in each user

In [12]:
#select negative post then groupby
neg_g = time.loc[time['negative_ny'] == 1].groupby(['userid'])
#negative post count
neg_c = neg_g['negative_ny'].count().reset_index()
neg_c.columns = ['userid', 'neg_count']

#all post count
post_c = time.groupby(['userid']).count().negative_ny.reset_index()
post_c.columns = ['userid', 'all_count']

per_df = pd.merge(neg_c, post_c, how = 'right', on = 'userid')
per_df['neg_count'].fillna(0, inplace=True)
#negative post percentage
per_df['neg_per'] = per_df['neg_count']/per_df['all_count']
per_df = per_df.drop('neg_count', axis=1)
per_df[1:10]

,userid,all_count,neg_per
1,0724fe854bd455061ba84efecdeff469,49,0.387755
2,088bf0ed841bcc366475b3975eb1b9d2,41,0.341463
3,0b12516100c16f779b152858619786ff,61,0.327869
4,0c5d06511ef79fba34da417e8c751fff,33,0.303030
5,0e3fdb89c7706b13891170ab2a9017c7,37,0.162162
6,1789589a6d2bba0f09d768f8f3445be5,51,0.470588
7,193165601dfad67e79ef52b2caf9fd9e,22,0.363636
8,1c1a61d11a10f9fa967f250095792db3,36,0.194444
9,1e84bfe1c9bbd549913aad1bc57c8878,34,0.176471


In [13]:
def count_percentage(category, OutputCol, OutputDf):
    cat_g = category.groupby(['userid'])
    #negative post count
    cat_c = cat_g['negative_ny'].count().reset_index()
    cat_c.columns = ['userid', 'cat_count']
    per_df = pd.merge(cat_c,OutputDf, how = 'right', on = 'userid')
    per_df['cat_count'].fillna(0, inplace=True)
    #negative post percentage
    per_df[OutputCol] = per_df['cat_count']/per_df['all_count']
    per_df = per_df.drop('cat_count', axis=1)
    return per_df

percentage of positive post in each user

In [14]:
cat_pos = time.loc[time['negative_ny'] == 2]
per_df = count_percentage(cat_pos,'pos_per', per_df)


percentage of mixed post in each user

In [15]:
cat_mix = time.loc[time['negative_ny'] == 3]
per_df = count_percentage(cat_mix,'mix_per', per_df)


percentage of neutral post in each user

In [16]:
#select mixed post then groupby
cat_neu = time.loc[time['negative_ny'] == 4]
per_df = count_percentage(cat_neu,'neu_per', per_df)
print(per_df.shape)
per_df[1:10]

(77, 6)


,userid,all_count,neg_per,pos_per,mix_per,neu_per
1,0724fe854bd455061ba84efecdeff469,49,0.387755,0.408163,0.020408,0.183673
2,088bf0ed841bcc366475b3975eb1b9d2,41,0.341463,0.439024,0.000000,0.219512
3,0b12516100c16f779b152858619786ff,61,0.327869,0.622951,0.000000,0.049180
4,0c5d06511ef79fba34da417e8c751fff,33,0.303030,0.575758,0.030303,0.090909
5,0e3fdb89c7706b13891170ab2a9017c7,37,0.162162,0.648649,0.054054,0.135135
6,10aef9f9c929fbfcffa58b68eda20dfb,4,0.000000,0.250000,0.000000,0.750000
7,1789589a6d2bba0f09d768f8f3445be5,51,0.470588,0.254902,0.019608,0.254902
8,193165601dfad67e79ef52b2caf9fd9e,22,0.363636,0.500000,0.090909,0.045455
9,1c1a61d11a10f9fa967f250095792db3,36,0.194444,0.555556,0.027778,0.222222


In [17]:
def ValencePercentage(Upper, Lower, OutputCol, OutputDf):
    cat_g = Upper.groupby(['userid'])
    #upper post count
    cat_c = cat_g['negative_ny'].count().reset_index()
    cat_c.columns = ['userid', 'cat_countUp']
    #lower post count
    cat_l = Lower.groupby(['userid'])
    cat_l2 = cat_l['negative_ny'].count().reset_index()
    cat_l2.columns = ['userid', 'cat_countLo']
    
    per_df = pd.merge(cat_c,cat_l2, how = 'outer', on = 'userid')
    per_df['cat_countUp'].fillna(0, inplace=True)
    per_df['cat_countLo'].fillna(0, inplace=True)
    #negative post percentage
    per_df[OutputCol] = per_df['cat_countUp']/per_df['cat_countLo']
    per_df = per_df.drop('cat_countUp', axis=1)
    per_df = per_df.drop('cat_countLo', axis=1)
    per_df = pd.merge(per_df,OutputDf, how = 'right', on = 'userid')
    return per_df

In [18]:
cat_upper = time.loc[time['negative_ny'] == 1]
cat_lower = time.loc[time['negative_ny'] == 2]
per_df = ValencePercentage(cat_upper,cat_lower, 'pos_neg', per_df)

In [19]:
#neutral/negative
cat_upper = time.loc[time['negative_ny'] == 4]
cat_lower = time.loc[time['negative_ny'] == 1]
per_df = ValencePercentage(cat_upper,cat_lower, 'neu_neg', per_df)
per_df[1:10]

,userid,neu_neg,pos_neg,all_count,neg_per,pos_per,mix_per,neu_per
1,0724fe854bd455061ba84efecdeff469,0.473684,0.950000,49,0.387755,0.408163,0.020408,0.183673
2,088bf0ed841bcc366475b3975eb1b9d2,0.642857,0.777778,41,0.341463,0.439024,0.000000,0.219512
3,0b12516100c16f779b152858619786ff,0.150000,0.526316,61,0.327869,0.622951,0.000000,0.049180
4,0c5d06511ef79fba34da417e8c751fff,0.300000,0.526316,33,0.303030,0.575758,0.030303,0.090909
5,0e3fdb89c7706b13891170ab2a9017c7,0.833333,0.250000,37,0.162162,0.648649,0.054054,0.135135
6,10aef9f9c929fbfcffa58b68eda20dfb,inf,0.000000,4,0.000000,0.250000,0.000000,0.750000
7,1789589a6d2bba0f09d768f8f3445be5,0.541667,1.846154,51,0.470588,0.254902,0.019608,0.254902
8,193165601dfad67e79ef52b2caf9fd9e,0.125000,0.727273,22,0.363636,0.500000,0.090909,0.045455
9,1c1a61d11a10f9fa967f250095792db3,1.142857,0.350000,36,0.194444,0.555556,0.027778,0.222222


Recode CESD data

In [ ]:
#match ces-d,big5 and swl

In [36]:
cesd = pd.read_csv('../data/dep.csv')
cesd.shape

(6561, 27)

In [37]:
#remove rows with empty answers
cesd_clean = cesd[['userid', 'q1', 'q2', 'q3', 'q4','q5', 'q6','q7', 'q8','q9','q10','q11','q12','q13','q14','q15','q16', 'q17','q18','q19','q20']]
cesd_clean = cesd_clean[cesd_clean.q1 != -1]
cesd_clean = cesd_clean[cesd_clean.q2 != -1]
cesd_clean = cesd_clean[cesd_clean.q3 != -1]
cesd_clean = cesd_clean[cesd_clean.q4 != -1]
cesd_clean = cesd_clean[cesd_clean.q5 != -1]
cesd_clean = cesd_clean[cesd_clean.q6 != -1]
cesd_clean = cesd_clean[cesd_clean.q7 != -1]
cesd_clean = cesd_clean[cesd_clean.q8 != -1]
cesd_clean = cesd_clean[cesd_clean.q9 != -1]
cesd_clean = cesd_clean[cesd_clean.q10 != -1]
cesd_clean = cesd_clean[cesd_clean.q11 != -1]
cesd_clean = cesd_clean[cesd_clean.q12 != -1]
cesd_clean = cesd_clean[cesd_clean.q13 != -1]
cesd_clean = cesd_clean[cesd_clean.q14 != -1]
cesd_clean = cesd_clean[cesd_clean.q15 != -1]
cesd_clean = cesd_clean[cesd_clean.q16 != -1]
cesd_clean = cesd_clean[cesd_clean.q17 != -1]
cesd_clean = cesd_clean[cesd_clean.q18 != -1]
cesd_clean = cesd_clean[cesd_clean.q19 != -1]
cesd_clean = cesd_clean[cesd_clean.q20 != -1]
cesd_clean.shape

(5833, 21)

In [38]:
#reverse coding
cesd_sum = cesd_clean.copy()
def reverse_coding(col):
    idxL1 = []
    idxL2 = []
    values = []
    for idx, val in enumerate(col):
        if val == 1:
            val = 4
            idxL1.append(idx)
            values.append(val)
    #print(idxL1)
        if val == 4 and idx not in idxL1:
            val = 1
            values.append(val)
        if val == 2:
            val = 3
            values.append(val)
            idxL2.append(idx)
        if val == 3 and idx not in idxL2:
            val = 2
            values.append(val)
    return values

recode_q4 = reverse_coding(cesd_sum['q4'])
recode_q8 = reverse_coding(cesd_sum['q8'])
recode_q12 = reverse_coding(cesd_sum['q12'])
recode_q16 = reverse_coding(cesd_sum['q16'])
recode_q4[0:10]

[1, 4, 4, 1, 1, 2, 1, 1, 3, 2]

In [39]:
cesd_sum['q4_recode'] = recode_q4
cesd_sum['q8_recode'] = recode_q8
cesd_sum['q12_recode'] = recode_q12
cesd_sum['q16_recode'] = recode_q16
cesd_sum = cesd_sum.drop(['q4','q8','q12','q16'], axis=1)

According to the original cesd scale, score is ranging from 0,1,2,3, our scores: 1,2,3,4, therefore, we shhould deduct 1 from each cell
https://perinatalweb.org/themes/wapc/assets/docs/center_for_epidemiological_studies.pdf

In [40]:
cesd_sum.iloc[:,1:21] = cesd_sum.iloc[:,1:21].applymap(lambda x: x-1)

cesd_sum['cesd_sum'] = cesd_sum.iloc[:,1:21].sum(axis = 1)
cesd_sum[1:5]
cesd_sum.to_csv('../important_data/adjustedCESD.csv')

In [41]:
cesd_sum.describe()

,q1,q2,q3,q5,q6,q7,q9,q10,q11,q13,...,q15,q17,q18,q19,q20,q4_recode,q8_recode,q12_recode,q16_recode,cesd_sum
count,5833.000000,5833.00000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,...,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000,5833.000000
mean,1.112978,0.92611,1.240357,1.659866,1.424310,1.508143,1.151209,1.091377,1.493400,1.227842,...,0.874679,0.753472,1.593005,1.275501,1.472827,1.442140,1.486028,1.391222,1.431167,26.331219
std,0.987972,1.02516,1.099867,1.028334,1.068057,1.029059,1.117009,1.048735,1.106471,1.021225,...,0.938013,0.968068,1.003918,1.082523,1.034218,1.068369,1.033983,0.950347,0.979962,12.981020
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000
50%,1.000000,1.00000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,26.000000
75%,2.000000,2.00000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,36.000000
max,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,60.000000


In [26]:
cesd_s = cesd_sum[['userid','cesd_sum']]
cesd_s[0:4]

,userid,cesd_sum
0,a54988e326d76a28f52a4b82d0efcd5d,10
1,6ef14eaaa9afbea3d61ede82103000d6,60
3,6ef14eaaa9afbea3d61ede82103000d6,37
4,036ccb0912e67c8f27f04926049d7c5c,6


In [27]:
all_df = pd.merge(per_df, cesd_s, how = 'left', on = 'userid')
all_df = all_df[np.isfinite(all_df['all_count'])]
print(all_df.shape)
all_df2 = all_df.drop_duplicates(subset='userid', keep='first', inplace=False)
print(all_df2.shape)

(101, 9)
(77, 9)


Description of the dataset

In [28]:
all_df2.describe()

,neu_neg,pos_neg,all_count,neg_per,pos_per,mix_per,neu_per,cesd_sum
count,76.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,inf,0.822350,53.233766,0.278628,0.489602,0.020914,0.203588,23.857143
std,NaN,1.134393,39.302101,0.125444,0.181292,0.025684,0.141555,11.758216
min,0.000000,0.000000,2.000000,0.000000,0.066667,0.000000,0.000000,2.000000
25%,0.350000,0.352941,30.000000,0.186441,0.360656,0.000000,0.105634,15.000000
50%,0.654762,0.563218,41.000000,0.295455,0.476190,0.017544,0.190476,25.000000
75%,1.157051,0.950000,59.000000,0.350877,0.600000,0.030303,0.264706,33.000000
max,inf,7.500000,206.000000,0.583333,1.000000,0.097561,0.750000,48.000000


Correlation of all variables

In [29]:
all_df2[1:9].corr()

,neu_neg,pos_neg,all_count,neg_per,pos_per,mix_per,neu_per,cesd_sum
neu_neg,1.000000,0.038779,0.050967,-0.363416,-0.130269,-0.167851,0.696284,-0.076396
pos_neg,0.038779,1.000000,0.555024,0.858079,-0.394014,0.002956,-0.287620,0.175189
all_count,0.050967,0.555024,1.000000,0.712769,0.319395,-0.254147,-0.638715,0.141019
neg_per,-0.363416,0.858079,0.712769,1.000000,0.025987,0.143879,-0.681498,0.163743
pos_per,-0.130269,-0.394014,0.319395,0.025987,1.000000,0.341238,-0.737733,-0.167137
mix_per,-0.167851,0.002956,-0.254147,0.143879,0.341238,1.000000,-0.461713,-0.605578
neu_per,0.696284,-0.287620,-0.638715,-0.681498,-0.737733,-0.461713,1.000000,0.091442
cesd_sum,-0.076396,0.175189,0.141019,0.163743,-0.167137,-0.605578,0.091442,1.000000


Here we see that the percentage of mixed post has a high correlation with cesd score, but it might be biased because the number of mixed post is only 88
Belows shows the stats of people have mixed posts
only 44 people have mixed posts, among 88 posts, 1/8 of the posts belong to the same person

In [30]:
mixed = time.loc[time['negative_ny'] == 3]
cat_g = mixed.groupby(['userid'])
#negative post count
cat_c = cat_g['negative_ny'].count().reset_index()
cat_c.columns = ['userid', 'cat_count']
cat_c.describe()


,cat_count
count,44.000000
mean,2.000000
std,1.765299
min,1.000000
25%,1.000000
50%,1.500000
75%,2.000000
max,11.000000
